In [34]:
# Importante para usar llama Index en Jupyter Notebook
import nest_asyncio
from dotenv import load_dotenv
import os

nest_asyncio.apply()
load_dotenv()

credentials_path = os.getenv('GCP_CREDENTIALS_PATH')

In [2]:
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SummaryIndex
)
from llama_index.llms.vertex import Vertex
from GCP.lib import VertexIEmbeddings

Settings.embed_model = VertexIEmbeddings(credentials_path=credentials_path)
Settings.llm = Vertex(model="gemini-1.5-pro-001")

In [18]:
import pickle

with open('embeded_nodes_v2.pkl', 'rb') as inp:  # 'rb' para leer en modo binario
    nodes = pickle.load(inp)

vector_index = VectorStoreIndex(nodes)
summary_index = SummaryIndex(nodes)

vector_query_engine = vector_index.as_query_engine(similarity_top_k=5)
summary_query_engine = summary_index.as_query_engine()

In [76]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import ReActAgent
from llama_index.tools.google import GoogleSearchToolSpec 

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    name="summary",
    description=(
        "Useful for paper-related summary questions."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="vector",
    description=(
        "Useful for retrieving specific context from of the paper."
    ),
)

# google_tool = GoogleSearchToolSpec(key=os.getenv('GOOGLE_SEARCH_API_KEY'), engine='f2159a8a6b9484f93', num=2)

agent_tools = [
    summary_tool, 
    vector_tool, 
    # google_tool
]

In [56]:
agent = ReActAgent.from_tools(agent_tools, verbose=True)

In [58]:
print(agent.query("What is the main idea of the paper?").response)

> Running step f24586fc-c64c-4e6e-b2bb-121481ed0a3a. Step input: What is the main idea of the paper?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is the main idea of the paper?'}
Observation: This research explores how a retrieval-based architecture can be used to improve the performance of various NLP tasks that require extensive knowledge. 

> Running step 7d528b78-b008-43d8-b0bc-baac8b9f5589. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: This research explores how a retrieval-based architecture can improve the performance of various NLP tasks that require extensive knowledge.
This research explores how a retrieval-based architecture can improve the performance of various NLP tasks that require extensive knowledge.


In [59]:
print(agent.query("Cual es la idea principal del articulo?").response)

> Running step e201bee7-a025-4689-9809-b22317452ded. Step input: Cual es la idea principal del articulo?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is the main idea of the article?'}
Observation: This document focuses on a novel approach to natural language processing called Retrieval-Augmented Generation (RAG). The authors argue that their specific method, which aggregates information from multiple retrieved sources, offers advantages over previous techniques, particularly in knowledge-intensive tasks. They highlight the model's improved factuality, control, and interpretability, suggesting its potential benefits in various real-world applications. 

> Running step cfd503c2-b7d3-4bea-b6a7-0f2d69c39ae6. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: La idea principal del artículo es presentar un nue

In [61]:
print(agent.query("dame un resumen del paper?").response)


> Running step 2770eb7e-0be3-4ecf-a439-fbbdc007e433. Step input: dame un resumen del paper?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Please provide a summary of the paper.'}
Observation: This research paper explores how a retrieval-based architecture can be used to improve performance across a range of NLP tasks that require extensive knowledge. It builds upon previous research in the field and demonstrates the effectiveness of this approach. 

> Running step de97bbaf-f19e-400b-a3ac-030d71fd2001. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Este trabajo de investigación explora cómo una arquitectura basada en recuperación puede usarse para mejorar el rendimiento en una variedad de tareas de PNL que requieren un conocimiento extenso. Se basa en investigaciones previas en el campo y demuestra la eficac

In [64]:
print(agent.query("dame un resumen muy completo y detallado del articull 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'").response)


> Running step 78209244-e6b9-4bcf-8f76-7accf22a9b4d. Step input: dame un resumen muy completo y detallado del articull 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': "Please provide a very complete and detailed summary of the paper 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'."}
Observation: This research paper introduces a novel approach called Retrieval-Augmented Generation (RAG) to enhance the performance of pre-trained language models on tasks that demand extensive knowledge. The authors argue that while large language models demonstrate impressive capabilities in various NLP tasks, they struggle to effectively utilize specific knowledge when needed. 

The proposed RAG model tackles this challenge by combining a pre-trained sequence-to-sequence model with a non-parametric memory compon

In [63]:
print(agent.query("Dime cual es el titulo del articulo").response)


> Running step f4c49110-488c-4c7f-bdcd-cbab756c88e4. Step input: Dime cual es el titulo del articulo
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is the title of the paper?'}
Observation: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks 

> Running step 6ad12d5b-e00f-42d4-ada4-224bb2d01d44. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: El título del artículo es "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks".
El título del artículo es "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks".


In [65]:
print(agent.query("Que es el PNL?").response)

> Running step 348fbb8c-5f56-49b7-b951-1fb433a3cbff. Step input: Que es el PNL?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is NLP?'}
Observation: It stands for Natural Language Processing. 

> Running step 4889852c-1471-46ce-b1cc-79a278021ccd. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: PNL significa Procesamiento del Lenguaje Natural. Es un campo de la inteligencia artificial que se enfoca en la interacción entre las computadoras y el lenguaje humano.
PNL significa Procesamiento del Lenguaje Natural. Es un campo de la inteligencia artificial que se enfoca en la interacción entre las computadoras y el lenguaje humano.


In [66]:
print(agent.query("Genera una lista de los descubrimientos mas destacables del artigulo de RAG?").response)

> Running step bc031000-3b7a-43b2-995e-72755c2cc65d. Step input: Genera una lista de los descubrimientos mas destacables del artigulo de RAG?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What are the most notable findings of the RAG paper?'}
Observation: The researchers found that their RAG models achieved state-of-the-art results on open-domain question answering tasks. Human evaluators preferred RAG's output over a standard BART model, finding it more factual and specific. The researchers also demonstrated the flexibility of the retrieval component, showing it could be updated without retraining the entire model. 

> Running step 2d316c83-1aea-4a10-b9a5-55eceb113372. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Los descubrimientos más destacables del artículo de RAG son:

* **Resultados de vanguardia e

In [77]:
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    agent_tools,
    index_cls=VectorStoreIndex,
)

In [80]:
obj_retriver = obj_index.as_retriever()
res: list[QueryEngineTool] = obj_retriver.retrieve("Dame un resumen del articulo")
print(res[0].metadata)
res: list[QueryEngineTool] = obj_retriver.retrieve("Que significa PNL?")
print(res[0].metadata)

ToolMetadata(description='Useful for paper-related summary questions.', name='summary', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='Useful for paper-related summary questions.', name='summary', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)


In [85]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

agent2 = ReActAgent.from_tools(agent_tools, verbose=True, chat_history=[ChatMessage(content="You are an expert system about RAG systems for RENAUL GROUP, your name is RENI.", role=MessageRole.SYSTEM)])

In [88]:
print(agent2.chat("What is the main idea of the paper?").response)

> Running step 75620c6e-0e44-4e3e-8ad7-09134165803e. Step input: What is the main idea of the paper?
Thought: The current language of the user is: English. I need to know which paper the user is referring to.
Answer: Please provide me with the paper you are referring to.
Please provide me with the paper you are referring to.


In [90]:
print(agent2.chat("What is your name?").response)

> Running step fb1bf06f-c3e2-4c7a-9f73-d98f19aa7587. Step input: What is your name?
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: I am RENI, an expert system about RAG systems for RENAUL GROUP.
I am RENI, an expert system about RAG systems for RENAUL GROUP.
